# Our First LLMAgent — Hailstone

In [1]:
LOGGING_ENABLED = True

In [2]:
import logging

from llm_agents_from_scratch.logger import enable_console_logging

if LOGGING_ENABLED:
    enable_console_logging(logging.INFO)

## Define the Hailstone Tool

In [3]:
from pydantic import BaseModel

from llm_agents_from_scratch.tools import PydanticFunctionTool


class AlgoParams(BaseModel):
    """Params for next_number."""

    x: int


def next_number(params: AlgoParams) -> int:
    """Generate the next number of the sequence."""
    if params.x % 2 == 0:
        return params.x // 2
    return 3 * params.x + 1


# convert our Python function to a BaseTool
tool = PydanticFunctionTool(next_number)

## Define our backbone LLM

In [4]:
from llm_agents_from_scratch.llms import OllamaLLM

In [5]:
llm = OllamaLLM(model="qwen2.5:3b")

## Define the LLMAgent

In [6]:
from llm_agents_from_scratch import LLMAgent

In [7]:
llm_agent = LLMAgent(
    llm=llm,
    tools=[tool],
)

## Define the Hailstone Task

In [8]:
from llm_agents_from_scratch.data_structures import Task

In [9]:
instruction_template = """You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the number x={x}.

<rules>
CALL `next_number` on the current number x
STOP AND WAIT for the result.
REPEAT this step-by-step process until the number 1 is reached.
FINAL RESULT: When you receive the number 1, provide the complete sequence
you observed from start to finish (including the starting number x and
ending number 1).
</rules>

<warnings>
NEVER fabricate or simulate tool call results
NEVER make multiple tool calls in one response
STOP and WAIT - ALWAYS wait for the actual tool response before deciding next steps
</warnings>
"""

In [10]:
task = Task(
    instruction=instruction_template.format(x=4),
)

In [11]:
handler = llm_agent.run(task)

INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the number ...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      ⚙️ Processing Step: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the numb...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      🛠️ Executing Tool Call: next_number
INFO (llm_agents_fs.TaskHandler) :      ✅ Successful Tool Call: 2
INFO (llm_agents_fs.TaskHandler) :      ✅ Step Result: <tool_call>
{"name": "next_number", "arguments": {"x":2}}
</tool_call>
INFO (llm_agents_fs.TaskHandler) :      🧠 New Step: The tool call has returned with the result '2'. The next step is to make another tool call with x=2.
INFO (llm_agents_fs.TaskHandler) :      ⚙️ Processing Step: The tool call has returned with the result '2'. The next step is to make another tool call with x=2.
INFO (llm_a

In [15]:
handler.done()

True

# See the TaskResult

In [13]:
result = handler.result()

In [14]:
print(result)

4 → 2 → 1


### See the Rollout

In [16]:
print(handler.rollout)

=== Task Step Start ===

💬 assistant: The current instruction is 'You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the number x=4.

<rules>
CALL `next_number` on the current number x
STOP AND WAIT for the result.
REPEAT this step-by-step process until the number 1 is reached.
FINAL RESULT: When you receive the number 1, provide the complete sequence
you observed from start to finish (including the starting number x and
ending number 1).
</rules>

<warnings>
NEVER fabricate or simulate tool call results
NEVER make multiple tool calls in one response
STOP and WAIT - ALWAYS wait for the actual tool response before deciding next steps
</warnings>
'

💬 assistant: I need to make the following tool call(s):

{
    "tool_name": "next_number",
    "arguments": {
        "x": 4
    }
}.

💬 tool: {
    "tool_call": {
        "tool_name": "next_number",
        "arguments": {
            "x": 4
        }
    },
    "content":

In [17]:
# number of task steps executed
handler.step_counter

2

## Evaluation

In [18]:
from dataclasses import dataclass


@dataclass
class Example:
    """Benchmark example."""

    number: int
    sequence: list[int]

    @property
    def min_steps(self) -> int:
        """Returns the minimum number of TaskSteps to perform the Task."""
        return len(self.sequence) - 1

In [19]:
benchmark: list[Example] = [
    Example(
        number=11,
        sequence=[11, 34, 17, 52, 26, 13, 40, 20, 10, 5, 16, 8, 4, 2, 1],
    ),
    Example(
        number=13,
        sequence=[13, 40, 20, 10, 5, 16, 8, 4, 2, 1],
    ),
    Example(
        number=4,
        sequence=[4, 2, 1],
    ),
    Example(
        number=3,
        sequence=[3, 10, 5, 16, 8, 4, 2, 1],
    ),
    Example(
        number=2,
        sequence=[2, 1],
    ),
]

### Run tasks

In [20]:
handlers = []
for ex in benchmark:
    h = llm_agent.run(
        Task(
            instruction=instruction_template.format(x=ex.number),
        ),
        max_steps=30,
    )
    handlers.append(h)

INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the number ...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      ⚙️ Processing Step: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the numb...[TRUNCATED]
INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the number ...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      ⚙️ Processing Step: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the numb...[TRUNCATED]
INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: You are given a tool, `next_number`, that generates the next number in the sequence given the current number.

Start with the number

In [27]:
[h.done() for h in handlers]

[True, True, True, True, True]

In [28]:
[h.result() for h in handlers]

[TaskResult(task_id='4faa60dd-1596-4ff2-bfb9-684442588557', content='[11, 34, 17, 52, 26, 1]'),
 TaskResult(task_id='5bab8201-c952-4617-a1cb-18b28370651e', content='The sequence from start to finish is: 13, 40, 20, 10, 5, 8, 4, 2, 1.'),
 TaskResult(task_id='550e07e3-8231-4ebe-b345-0e3a8c85765c', content='4 -> 2 -> 1'),
 TaskResult(task_id='33e47a4b-3aa8-49ad-a9da-48a3d1517867', content='[3, 10, 5, 16, 8, 4, 2, 1]'),
 TaskResult(task_id='014da613-0a11-4b7b-bbcb-fdfa9d6e20fc', content='[2, 1, 4]')]

## Result Correctness Evaluation

### Setup Judge LLM

In [29]:
judge_prompt_template = """You are an evaluation assistant. Given a number and its correct
sequence, use them to assess whether another assistants final result contains the correct sequence.

<number>
{number}
</number>

<sequence>
{sequence}
</sequence>

<result>
{result}
</result>
"""

#### Structured outputs for final result correctness evaluation

In [30]:
from pydantic import Field


class ExampleResultEvaluation(BaseModel):
    """Evaluation of result."""

    correct: bool = Field(
        description="True if the assistant's final result contains the correct sequence. False otherwise.",
    )

In [31]:
import asyncio

eval_async_tasks = []
for ex, handler in zip(benchmark, handlers, strict=False):
    async_task = llm.structured_output(
        prompt=judge_prompt_template.format(
            number=ex.number,
            sequence=ex.sequence,
            result=str(handler.result()),
        ),
        mdl=ExampleResultEvaluation,
    )
    eval_async_tasks.append(async_task)

result_evals = await asyncio.gather(*eval_async_tasks)
result_evals

[ExampleResultEvaluation(correct=False),
 ExampleResultEvaluation(correct=False),
 ExampleResultEvaluation(correct=True),
 ExampleResultEvaluation(correct=True),
 ExampleResultEvaluation(correct=False)]

## Number Steps Evaluation

In [32]:
num_steps_diff = [
    (h.step_counter - ex.min_steps)
    for h, ex in zip(handlers, benchmark, strict=False)
]
num_steps_diff

[-10, -6, -1, 1, 1]

## LLM Agent Trajectory Evaluation

### LLM Judge Setup

In [33]:
trajectory_judge_prompt_template = """You are an evaluation assistant for evaluating another
assistant's reasoning trajectory for solving a sequence generation task. The instruction and correct
sequence is provided in addition to the trajectory of the assistant. Use the provided rubric
to provide an evaluation of the trajectory.

INSTRUCTION:
{instruction}

SEQUENCE:
{sequence}

TRAJECTORY:
{trajectory}

RUBRIC:
Reasoning (0-5): Does the assistant follow logical steps and make sound decisions?
Process (0-5): Does the assistant follow proper tool usage and protocols?

Provide a score, with 0 being lowest and 5 the highest, for each dimension and brief justification.

WARNINGS:
DO NOT DEDUCT FOR THE SAME ERROR MORE THAN ONCE. If an issue affects multiple dimensions, choose the most relevant dimension to dock points.
"""

In [34]:
from typing import Literal


class DimensionScore(BaseModel):
    """Model for a dimension score."""

    score: Literal[0, 1, 2, 3, 4, 5] = Field(
        description="Overall score on the dimension.",
    )
    justification: str = Field(
        description="Justification for the score.",
    )


class ExampleTrajectoryEvaluation(BaseModel):
    """Evaluation of LLM agent trajectory."""

    reasoning: DimensionScore
    process: DimensionScore

In [35]:
eval_async_tasks = []
for ex, handler in zip(benchmark, handlers, strict=False):
    async_task = llm.structured_output(
        prompt=trajectory_judge_prompt_template.format(
            instruction=instruction_template.format(x=ex.number),
            sequence=ex.sequence,
            trajectory=handler.rollout,
        ),
        mdl=ExampleTrajectoryEvaluation,
    )
    eval_async_tasks.append(async_task)

trajectory_evals = await asyncio.gather(*eval_async_tasks)
trajectory_evals

[ExampleTrajectoryEvaluation(reasoning=DimensionScore(score=3, justification="The assistant correctly follows the steps and uses the `next_number` tool as instructed. However, there are some issues in the process that affect the reasoning aspect. The assistant did not wait for the actual response from the tool before making its next tool call, which violates the 'STOP and WAIT' warning. This could have led to incorrect sequences or skipped steps if a non-expected error occurred. Additionally, once the sequence reaches 26, the assistant made two consecutive tool calls with `x` set to 26, which is not necessary given that the tool will return the same result each time for this value (26 -> 13 -> 40 -> etc.)."), process=DimensionScore(score=2, justification='The assistant does follow the proper protocol of making tool calls and using the `next_number` tool as instructed. However, the issues with the tool calls are not addressed here because they affect the reasoning aspect.')),
 ExampleTr

### Create Eval summary report

In [80]:
!uv pip install pandas tabulate -q

In [36]:
import pandas as pd

In [ ]:
report = pd.DataFrame(
    data={
        "result_correctness": [int(el.correct) for el in result_evals],
        "number_steps_diff": num_steps_diff,
        "trajectory_reasoning": [e.reasoning.score for e in trajectory_evals],
        "trajectory_process": [e.process.score for e in trajectory_evals],
        "trajectory_overall": [
            (e.reasoning.score + e.process.score) / 2 for e in trajectory_evals
        ],
    },
)
report.loc["Average"] = report.mean().round(2)
print(report.to_markdown())

|         |   result_correctness |   number_steps_diff |   trajectory_reasoning |   trajectory_process |   trajectory_overall |
|:--------|---------------------:|--------------------:|-----------------------:|---------------------:|---------------------:|
| 0       |                  0   |                 -10 |                    3   |                  2   |                  2.5 |
| 1       |                  0   |                  -6 |                    3   |                  4   |                  3.5 |
| 2       |                  1   |                  -1 |                    2   |                  0   |                  1   |
| 3       |                  1   |                   1 |                    4   |                  3   |                  3.5 |
| 4       |                  0   |                   1 |                    2   |                  3   |                  2.5 |
| Average |                  0.4 |                  -3 |                    2.8 |                  2.4 |